# Generate point-in-time best execution data using dedicated service - Python

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **a list of trades** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window
* plot the request result and export data

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.helpers.token_helpers as token_helpers
import systemathics.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request

#### 3.1 Input trades import
We assume our trades are in a *csv file* that we imported in the same folder as the current data sample.

We aim to to ensure that it was the best possible execution for each and every trade. To do so, we have to retrieve the *trades as a market snapshot* and mark out the *individual trades*.

The following code snippets import *individual trades* from the *csv file*:

In [ ]:
df_input = pd.read_csv("input_trades.csv")
df_input = df_input.sort_values(by="Timestamp")
df_input

#### 3.2 Instrument & time window selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

# set the time window in minutes to delimit each input trade
window = 5

#### 3.3 Time period delimitation

In [ ]:
time_format = '%Y-%m-%dT%H:%M:%S.%fZ'

# create date interval (we are using Google date format)
first_date = datetime.strptime(df_input['Timestamp'][0], time_format).date()
last_date = datetime.strptime(df_input['Timestamp'][df_input.index[-1]], time_format).date()

date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = first_date.year, month = first_date.month, day = first_date.day), 
    end_date = date.Date(year = last_date.year, month = last_date.month, day = last_date.day)
)

# create time interval (we are using Google time format)
# UTC time zone
start_time = datetime.strptime(df_input['Timestamp'][0], time_format) - timedelta(minutes = window)
end_time = datetime.strptime(df_input['Timestamp'][df_input.index[-1]], time_format) + timedelta(minutes = window)

time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = start_time.hour, minutes = start_time.minute, seconds = start_time.second), 
    end_time = timeofday.TimeOfDay(hours = end_time.hour, minutes = end_time.minute, seconds = end_time.second)
)

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate the tick trades request
identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_trades.TickTradesRequest(
    identifiers = [identifier],
    constraints = constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick trades service
        service = tick_trades_service.TickTradesServiceStub(channel)
        
        # process the tick trades request
        trades = []
        metadata = [('authorization', token)]
        for trade in service.TickTrades(request=request, metadata=metadata):
            if trade.trade:
                trades.append(trade.trade)

    # keep only trades, delete instrument mapping (1st element retrieved)
    # mapping is not used as only one instrument identifier is requested
    del trades[0]
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved trades count:

In [ ]:
print('Total trades retrieved: {0}'.format(len(trades)))

### Step 4: Visualize data

#### 4.1 Retrieve data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates = [(datetime.fromtimestamp(t.time_stamp.seconds) + timedelta(microseconds = t.time_stamp.nanos/1000)) for t in trades]
prices = [t.price for t in trades]
sizes = [t.size for t in trades]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d = {'Date': dates, 'Price': prices, 'Size': sizes}
df_market = pd.DataFrame(data=d)
df_market

#### 4.2 Plot trades with matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
# Add labels to the plot
ax.plot('Date', 'Price', data=df_market, marker='', color='blue', alpha = 0.5, linewidth=2, label="Price")
timestamps = [datetime.strptime(elt, time_format) for elt in df_input['Timestamp']]
ax.plot(timestamps, df_input['Price'], color='blue', linestyle='None', marker='D', markersize=10, label="Input trade price")

# twin x-axis for two different y-axis
ax2=ax.twinx()
ax2.plot('Date', 'Size', data=df_market, marker='', color='red', alpha = 0.25, linewidth=2, label="Size")
ax2.plot(timestamps, df_input['Size'], color='red', linestyle='None', marker='D', markersize=10, label="Input trade size")

# set graph title and axis label
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="blue",fontsize=14)
ax2.set_ylabel("Size",color="red", fontsize=14)
plt.title('Input trade within market snapshot for {0}-{1}'.format(ticker, exchange))
ax.legend(loc='upper left')
plt.legend(loc='upper right')
plt.show()

#### 4.3 Plot trades with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

timestamps, prices, sizes, flags = [], [], [], []

for i in df_market.index:
    keep = False
    flag = None
    marketstamp = df_market['Date'][i]
    
    for j in df_input.index:
        timestamp = datetime.strptime(df_input['Timestamp'][j], time_format)
        start = timestamp - timedelta(minutes = window)
        end = timestamp + timedelta(minutes = window)
        
        if (marketstamp > start  and  marketstamp < end):
            keep = True
            if ((marketstamp == timestamp) and (df_market['Price'][i] == df_input['Price'][j]) and (df_market['Size'][i] == df_input['Size'][j])):
                flag = 'INPUT'
                
    if keep:
        timestamps.append(marketstamp) 
        prices.append(df_market['Price'][i])
        sizes.append(df_market['Size'][i])
        flags.append(flag)

# TODO
# # create figure with secondary y-axis: price and size
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], name='Price'), secondary_y=False)
# fig.add_trace(go.Scatter(x=df['Date'], y=df['Size'], name="Size", opacity = 0.25), secondary_y=True)

# # annotate input trade
# fig.add_annotation( x=timestamp, y=price, xref="x", yref="y", text='{0} @ {1} | {2}'.format(size, price, timestamp), showarrow=True, font=dict(family="Courier New, monospace", size=16,color="white"),
#         align="center", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="green", ax=20, ay=-100, bgcolor="green", opacity=0.8)

# # set graph layout
# fig.update_xaxes(title_text = 'Date')
# fig.update_layout(title = 'Input trades within market snapshot for {0}-{1}'.format(ticker,exchange))
# fig.layout.yaxis2.showgrid=False
# fig.show()

#### 4.4 Export market snapshot file

In [ ]:
# create a pandas dataframe with: timestamps, trades prices, sizes and flags
d = {'Timestamp': timestamps, 'Price': prices, 'Size': sizes, 'Flag': flags}
df = pd.DataFrame(data=d)

In [ ]:
# export csv file
df.to_csv('Export/{0}-{1}_bestex_pit.csv'.format(ticker,exchange), index=False)